# Facial Expression Recognition

This is an attempt to detect facial expressions.

See: https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge

The solution is using pre-weighted VGG16 from Keras library

In [1]:
%matplotlib inline
import pandas as pd
import os
import json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from scipy.misc import imresize, imsave

In [2]:
import csv
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from keras.utils import np_utils
source_size = (48,48)
target_size = (224,224)
cmap = plt.get_cmap('hot')

# Data Labels Defined
labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Read in data in an efficient manner
# We need to convert the black and white image data to an RGB image that the VGG16 model expects
# We accomplish the color conversion with the use of a color map

def load_data(filename, dataset=None):
    assert dataset in ['Training', 'PublicTest', 'PrivateTest']
    with open(filename, 'rb') as csvfile:
        reader = csv.reader(csvfile)
        header = reader.next()
        usages = set()
        i=0
        for row in tqdm_notebook(reader):
            emotion = int(row[0])
            usage = row[2]
            usages.add(usage)

            if usage != dataset: continue
                           
            image_data = imresize(pd.DataFrame(row[1].split(), dtype=int).values.reshape(*source_size),target_size)
            image_data = np.delete(cmap(image_data), 3, 2)
            #image_data = np.swapaxes(image_data, 3, 1)
            filename = "%s/%s/%s.png" % (dataset,emotion, i)
            dirname = os.path.dirname(filename)
            if not os.path.exists(dirname):
                os.makedirs(dirname,)           
            imsave(filename, image_data)
            i+=1

Using Theano backend.


In [ ]:
# Load the training data set
load_data('fer2013/fer2013.csv', dataset='Training')
load_data('fer2013/fer2013.csv', dataset='PublicTest')
load_data('fer2013/fer2013.csv', dataset='PrivateTest')

In [110]:
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, Lambda, Input, Flatten, Dropout
from keras.models import Model, load_model
from keras import optimizers
from keras.applications import vgg16

nb_classes = len(labels)
weights_file = "weights.h5"
target_size=(224,224)

vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))
def vgg_preprocess(x):
    x = x - vgg_mean
    return x[:, ::-1] # reverse axis rgb->bgr

def create_default_model():
    # build the model from scratch using VGG16 as a base
    model_vgg16 = vgg16.VGG16(weights='imagenet', include_top=False)
    model_vgg16.summary()
    for layer in model_vgg16.layers:
        layer.trainable = False
        
    input_layer = Input(shape=(3,224,224),name = 'image_input')
    preprocess_layer = Lambda(vgg_preprocess, input_shape=(3,224,224), output_shape=(3,224,224))(input_layer)
    #Use the generated model 
    output_vgg16 = model_vgg16(preprocess_layer)

    #Add the fully-connected layers 
    x = Flatten(name='flatten')(output_vgg16)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dropout(0.5)(x)
    x = Dense(4096, activation='softmax', name='fc2')(x)
    x = Dropout(0.5)(x)
    x = Dense(nb_classes, activation='softmax', name='predictions')(x)
    model = Model(input=input_layer, output=x)
    return model

def add_compiler(model, lr=0.001):
    model.compile(optimizer=optimizers.Adam(lr=lr), loss='categorical_crossentropy', metrics=['accuracy'])

def load_model_with_weights_if_available():
    # if weights exists on disk, then load it
    model = create_default_model()
    if os.path.exists(weights_file):
        #model.load_weights(weights_file)
        print("Model loaded from file %s" % weights_file)
    else:
        print("Model built from scratch")
        
    add_compiler(model)
    model.summary()
    return model

def get_batches(path, gen=ImageDataGenerator(), shuffle=True, batch_size=8, class_mode='categorical'):
        return gen.flow_from_directory(path, target_size=(224,224),
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)
    
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (len(ims.shape) == 4 and ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')    

In [105]:
model = load_model_with_weights_if_available()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 3, None, None) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 64, None, None)1792        input_5[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 64, None, None)36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, None, None)0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [106]:
batch_size=32
batches = get_batches("Training", batch_size=batch_size)
samples = get_batches("TrainingSample", batch_size=batch_size)
public_batches = get_batches("PublicTest", batch_size=batch_size)
private_batches = get_batches("PrivateTest", batch_size=batch_size)

Found 28709 images belonging to 7 classes.
Found 90 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [107]:
# MAIN TRAINING CYCLE
# Fit the model, and then save the weights to disk
nb_epoch = 5
N_test=10
samples = get_batches("TrainingSample", batch_size=10, shuffle=True)
N = samples.N
#N = 20
model.fit_generator(samples, N, nb_epoch, 
                    validation_data=public_batches, 
                    nb_val_samples=N_test)
#model.save_weights(weights_file)

Found 90 images belonging to 7 classes.
Epoch 1/5
90/90 [==============================] - 60s - loss: 1.9491 - acc: 0.1444 - val_loss: 1.9529 - val_acc: 0.0625
Epoch 2/5
90/90 [==============================] - 60s - loss: 1.9426 - acc: 0.2000 - val_loss: 1.9485 - val_acc: 0.1250

In [103]:
# Check Accuracy of the test data sets
loss, acc = model.evaluate_generator(public_batches, 100)
print("Public Test Loss: %.4f, Accuracy: %.4f" % (loss, acc))
loss, acc = model.evaluate_generator(private_batches, 100)
print("Private Test Loss: %.4f, Accuracy: %.4f" % (loss, acc))

Public Test Loss: 1.9205, Accuracy: 0.1328
Private Test Loss: 1.9240, Accuracy: 0.1016


In [205]:
# predict private set results and save submission file to disk
results = model.predict(private_batches)
values = np.argmax(results, axis=1)
with open("submission_private.csv", "wb") as fp:
    for x in values:
        fp.write("%d\n" % x)
    fp.close()